In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random


from datetime import datetime

from itertools import combinations

from matplotlib.backends.backend_pdf import PdfPages

from numpy import absolute
from numpy import arange
from numpy import mean
from numpy import std
from numpy import sqrt

from scipy.stats import pearsonr, linregress
import scipy as sp

from sklearn.linear_model import LassoCV
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

from sklearn.ensemble import RandomForestRegressor

import joblib

# Section 1: Code for Creating and Fitting the RandomForest Model

In [ ]:
def read_KP_data(path_to_excel = '', na_val = 0):
    """
    Code to create a dataframe from the excel sheet
    Inputs:
        -path_to_excel: the path to the excel sheet with community information, presented as a 
         binary matrix of strain presence/absence
        -na_val: the value used to replace empty cells in the excel file. Needed only to prevent
         errors from excel formatting
         
    Output:
        -final_data: a pandas dataframe presented as a binary matrix of strain presence/absence
    """
    datasheet = pd.read_excel(path_to_excel).fillna(na_val)
    
    final_data = datasheet.drop(datasheet.columns[[0,-2]], axis = 1) 
    #since we fit our data on a log-10 scale, we drop the column that contains the raw CFU counts
    #this is to make all downstream code easier
    
    return final_data

In [ ]:
def split_data_train_test(data, size_eval=10):
    """
    Code to split a pandas dataframe into training and testing datasets
    Inputs:
        -data: the pandas dataframe created from read_KP_data
        -size_eval: the number of datapoints to hold in the test set
        
    Outputs:
        -Xtrain: a pandas dataframe consisting of communinities to be used as the training set
         presented as a binary matrix of strain presence/absence
        -Ytrain: a pandas series consisting of the response variable (log CFU) to be used in
         the training set
        -Xeval: a pandas dataframe consisting of communinities to be held out as the test set
         presented as a binary matrix of strain presence/absence
        -Yeval: a pandas series consisting of the response variable (log CFU) to be held out in
         the test set
    """
    n = data.shape[0]
    train_indices = random.sample(range(n), n - size_eval) #randomly sampling training set communities
    test_indices = [z for z in range(n) if z not in train_indices]
    
    Xtrain = data.iloc[train_indices,:-1] 
    Ytrain = data.iloc[train_indices,-1:]

    Xeval = data.iloc[test_indices, :-1]
    Yeval = data.iloc[test_indices, -1:]
    
    return Xtrain, Ytrain, Xeval, Yeval

In [ ]:
def build_RF_model(data, size_eval = 10, num_runs = 100, plot_significance_scores = False):
    """
    Code to build and train the RandomForest Model
    Inputs:
        -data: the pandas dataframe created from read_KP_data
        -size_eval: the size of the testing set to be held out from the data. Default to 10
        -num_runs: the number of times to split the dataset and fit a RandomForest to each split
         Default to 100
        -plot_significance_scores: a boolean value of whether to plot the significant scores, with their
         variance as determined from the model fits on data splits. Default to false
         
    Output:
        -model: the RF model fit on the entire dataset
    """
    
    #Part 1: splitting the data multiple times, and fitting a separate model each time
    #Purpose: a metric for understanding extent of uncertainty in significance scores
    
    sig_vectors = np.zeros((data.shape[1] - 1, num_runs)) #storing the significance scores from each run
    
    for i in range(num_runs):
        Xtrain, Ytrain, Xeval, Yeval =  split_data_train_test(data, size_eval)
        
        model = RandomForestRegressor(max_depth = 12, max_features= "sqrt", bootstrap=True, 
                                      oob_score=True, n_jobs= -1)
        #We found that a maximum depth of greater than 12 has no effect on the model
        model.fit(Xtrain, Ytrain)
        sig_vectors[:,i] = model.feature_importances_
        
    error_of_sig_scores = std(sig_vectors, axis=1, ddof=1) / sqrt(len(sig_vectors))
    
    #Part 2: Fit the entire data to the RF model
    Xtrain, Ytrain, _, _ =  split_data_train_test(data, 0) #size_eval=0 ensures all data is training
    model = RandomForestRegressor(max_depth = 12, max_features= "sqrt", bootstrap=True, 
                                      oob_score=True, n_jobs= -1)
    model.fit(Xtrain, Ytrain)
    
    if plot_significance_scores:
        plt.bar(range(data.shape[1] - 1), model.feature_importances_, 
                yerr=error_of_sig_scores)
        plt.ylabel('Significance Scores')
        plt.title("RandomForest Significant Scores")
        plt.show()
        
    return model

In [ ]:
def test_RF_model(model, validation_data, plot_predictions = False):
    """
    Code to validate the RF model on previously unseen dataset
    Inputs:
        -model: The RandomForest model from build_RF_model
        -validation_data: the pandas dataframe created from read_KP_data containing the previously
         unseen data
        -plot_predictions: a boolean value of whether to plot the predictions from the model. Default to False
        
    Output:
        -predictions: an array of the predicted log CFUs from the model
    """
    size_eval = validation_data.shape[0]
    _, _, X, Y =  split_data_train_test(validation_data, size_eval) #ensures all data is testing
    
    predictions = model.predict(X)
    
    Y = np.array(Y).reshape(-1,) #needed to calculate statistics
    
    pearson_correlation = pearsonr(predictions,Y)[0]
    r2_score = linregress(predictions,Y)[2] ** 2
    
    if plot_predictions:
        total_min = min(np.min(predictions), np.min(Y))
        total_max = max(np.max(predictions), np.max(Y))
        
        plt.scatter(predictions,Y)
        plt.plot([total_min, total_max], [total_min, total_max], 'k--') #y=x line, showing ideal predictions
        
        plt.xlabel('Log CFUs predicted by RandomForest')
        plt.ylabel('Measured Log CFUs')
        plt.suptitle("Predictive Power of RandomForest Model")
        plt.title("R^2: %0.4f" % r2_score)
        plt.show()
        
    return predictions
        

In [ ]:
# Parameters to run the RF model:
initial_communities_file = "./Initial_96_Communities.xlsx"
validation_communities_file = "./Validation_60_Communities.xlsx"

In [ ]:
def main():
    """
    Runs all of the code
    Input: none
    
    Outputs:
        -model: the resulting RandomForest model
    """
    initial_data = read_KP_data(initial_communities_file)
    validation_data = read_KP_data(validation_communities_file)
    model = build_RF_model(initial_data, plot_significance_scores = True)
    predictions = test_RF_model(model, validation_data, plot_predictions = True)
    
    return model

In [ ]:
model = main()

# Section 2: Exploring a Space of 1,000,000 Communities In-Silico

In [ ]:
def sample_random_communities(number, size=46):
    """
    Creates randomly assembled communities in-silico
    Inputs:
        -number: the number of communities to create
        -size: the number of binary digits for each community. Default to 46, because all of our
         communities have 46 strains
         
    Output:
        -communities: a numpy array of binary digits, containing synthetic communities represented
         by strain presence/absence
    """
    communities = []
    for i in range(number):
        communities.append(np.random.randint(2,size=size)) #generates an array of 0s/1s

    communities = np.unique(communities, axis=0) #gets rid of duplicate communities
    difference = number - communities.shape[0]
    while difference > 0: #regenerates new communities equal to the number of duplicate ones removed
        for i in range(difference):
            communities.append(np.random.randint(2,size=size))
                               
        communities = np.unique(communities, axis=0)
        difference = number - communities.shape[0]
        
        
    return communities

In [ ]:
def run_1000000_simulation(model):
    """
    Simulates 10^6 communities
    Input:
        -model: a trained RF model
        
    Outputd:
        -communities: the 10^6 communities generated from sample_random_communities
        -predictions: the RF model's predicted log CFUs for all communities
    """
    number = 1000000
    
    communities = sample_random_communities(number)
    
    predictions = model.predict(communities)
    
    plt.hist(predictions, bins=[3,4,5,6,7,8,9], histtype='bar')
    plt.xlabel('Log CFUs')
    plt.ylabel('Frequency')
    plt.title('Generation of 10^6 Communities In-Silico')
    plt.show()
    
    return communities, predictions

In [ ]:
simulated_comms, simulated_preds = run_1000000_simulation(model)


In [ ]:
simulated_comms_sizes = np.sum(simulated_comms, axis=1)
plt.hist(simulated_comms_sizes, histtype='bar')
plt.xlabel('Size of communities')
plt.ylabel('Frequency')
plt.title('Generation of 10^6 Communities In-Silico')
plt.show()

# Section 3: Loading in the RandomForest Model from Joblib File

This is the model used for all analysis

In [ ]:
presaved_model = joblib.load('./KP_RF.joblib')